# Imports

In [ ]:
%env CUDA_VISIBLE_DEVICES=0

In [ ]:
import torch
from torch import nn
from torchvision import models

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['figure.facecolor'] = 'white'
matplotlib.rcParams['figure.figsize'] = (15, 5)

In [ ]:
import pandas as pd
pd.options.display.max_columns = None

In [ ]:
%run ../utils/__init__.py
config_logging(logging.INFO)

# Debug model

## Feature extractor

In [ ]:
extractor = VisualFeatureExtractor('resnet152', pretrained=True)
extractor

In [ ]:
BATCH_SIZE = 7

In [ ]:
images = torch.rand(BATCH_SIZE, 3, 224, 224)
images.size()

In [ ]:
local_features, global_features = extractor(images)
local_features.size(), global_features.size()

## MLC

In [ ]:
mlc = MLC(fc_in_features=extractor.out_features)
mlc

In [ ]:
tags, semantic_features = mlc(global_features)
tags.size(), semantic_features.size()

## Co-attention

In [ ]:
embed_size = 512
hidden_size = 512
k = 10
bn_momentum = 0.1

In [ ]:
coatt = CoAttention(version='v4',
                    embed_size=embed_size,
                    hidden_size=hidden_size,
                    visual_size=extractor.out_features,
                    k=k,
                    momentum=bn_momentum,
                   )

In [ ]:
prev_hidden_states = torch.zeros(BATCH_SIZE, 1, hidden_size)
prev_hidden_states.size()

In [ ]:
ctx, other_a, other_b = coatt(global_features, semantic_features, prev_hidden_states)
ctx.size(), other_a.size(), other_b.size()

## Sentence model

In [ ]:
sentence_num_layers = 2

In [ ]:
sentence_model = SentenceLSTM(
    version='v1', embed_size=embed_size, hidden_size=hidden_size,
    num_layers=sentence_num_layers, dropout=0,
    momentum=bn_momentum)

In [ ]:
sentence_states = None

In [ ]:
topic, p_stop, hidden_states, sentence_states = sentence_model(ctx,
                                                               prev_hidden_states,
                                                               sentence_states)

In [ ]:
topic.size(), p_stop.size(), hidden_states.size(), sentence_states[0].size()

## Word model

In [ ]:
%run ../models/report_generation/coatt/lstm.py

In [ ]:
vocab_size = 1000

In [ ]:
word_model = WordLSTM(vocab_size=vocab_size,
                      embed_size=512,
                      hidden_size=512,
                      num_layers=1,
                      n_max=30)

In [ ]:
BATCH_SIZE = 2
topic = torch.rand(BATCH_SIZE, 1, 512)
captions = (torch.rand(BATCH_SIZE, 3, 19) * (vocab_size - 1)).long()

In [ ]:
word_model.train()
words = word_model(topic, captions[:, 0, :1])
words.size()

In [ ]:
words.argmax(-1)

In [ ]:
word_model.eval()
words = word_model(topic, None)
words.size()

In [ ]:
words

## Losses

In [ ]:
label = (torch.rand(BATCH_SIZE, 156) > 0.5)

In [ ]:
mse_criterion = nn.MSELoss()

In [ ]:
batch_tag_loss = mse_criterion(tags, label).sum()
batch_tag_loss

In [ ]:
prob_real = (torch.rand(BATCH_SIZE, ) > 0.5).long()

In [ ]:
batch_stop_loss = words_criterion(p_stop.squeeze(), prob_real).sum()
batch_stop_loss

## Full model

In [ ]:
%run ../models/report_generation/coatt/__init__.py

In [ ]:
VOCAB_SIZE = 1234

In [ ]:
model = CoAttModel(range(VOCAB_SIZE))
# model

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
BATCH_SIZE = 7
N_SENTENCES = 3
N_WORDS = 5
images = torch.rand(BATCH_SIZE, 3, 224, 224)
labels = (torch.rand(BATCH_SIZE, 14) > 0.5).long()
captions = (torch.rand(BATCH_SIZE, N_SENTENCES, N_WORDS) * (VOCAB_SIZE - 1)).long()
prob_real = torch.rand(BATCH_SIZE, N_SENTENCES).long()

In [ ]:
out = model(images, captions, prob_real)
len(out)

In [ ]:
words, tags, l1, l2 = out
words.size()

In [ ]:
_, l1, l2 = out
l1.type(), l2.type()

In [ ]:
prob_real.size()

In [ ]:
def get_step_fn(model, optimizer=None, training=True,
                lambda_tag=1,
                lambda_stop=1,
                lambda_word=1,
                clip=0,
               ):
    mse_criterion = nn.MSELoss()
    def step_fn(unused_engine, batch):
        images, _, labels, reports, gt_stops = batch
        
        # Forward pass
        words, tags, batch_stop_loss, batch_word_loss = model(images, reports, gt_stops)
        
        # Tags loss
        batch_tag_loss = mse_criterion(tags, labels.float()).sum()

        # Total loss
        batch_loss = lambda_tag * batch_tag_loss \
                     + lambda_stop * batch_stop_loss \
                     + lambda_word * batch_word_loss

        if optimizer:
            optimizer.zero_grad()

        if training:
            batch_loss.backward()

            if clip > 0:
                torch.nn.utils.clip_grad_norm(model.sentence_model.parameters(), clip)
                torch.nn.utils.clip_grad_norm(model.word_model.parameters(), clip)

            optimizer.step()
            
        return {
            'loss': batch_loss.detach(),
        }
    return step_fn

In [ ]:
step_fn = get_step_fn(model, optimizer)

In [ ]:
def get_step_fn(model, optimizer=None, training=True,
                lambda_tag=1,
                lambda_stop=1,
                lambda_word=1,
                clip=0,
               ):
    mse_criterion = nn.MSELoss()
    def step_fn(unused_engine, batch):
        images, _, labels, reports, gt_stops = batch
        
        # Forward pass
        tags, batch_stop_loss, batch_word_loss = model(images, label, captions, gt_stops)
        
        # Tags loss
        batch_tag_loss = mse_criterion(tags, labels.float()).sum()

        # Total loss
        batch_loss = lambda_tag * batch_tag_loss \
                     + lambda_stop * batch_stop_loss \
                     + lambda_word * batch_word_loss

        if optimizer:
            optimizer.zero_grad()

        if training:
            batch_loss.backward()

            if clip > 0:
                torch.nn.utils.clip_grad_norm(model.sentence_model.parameters(), clip)
                torch.nn.utils.clip_grad_norm(model.word_model.parameters(), clip)

            optimizer.step()
            
        return {
            'loss': batch_loss.detach(),
        }
    return step_fn

In [ ]:
step_fn(None, (images, None, labels, captions, prob_real))

## Their dataloader

In [ ]:
%run -n ../../../software/Medical-Report-Generation/utils/dataset.py

In [ ]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406),
                         (0.229, 0.224, 0.225))])

In [ ]:
new_data_dir = '/home/pdpino/software/Medical-Report-Generation/data/new_data'

In [ ]:
with open(f'{new_data_dir}/vocab.pkl', 'rb') as f:
    vocab = pickle.load(f)

In [ ]:
data_loader = get_loader(
    image_dir='/mnt/workspace/iu-x-ray/dataset/images',
    caption_json=f'{new_data_dir}/captions.json',
    file_list=f'{new_data_dir}/train_data.txt',
    vocabulary=vocab,
    transform=transform,
    batch_size=10,
    shuffle=False,
)

In [ ]:
dataloader = iter(data_loader)

In [ ]:
batch = next(dataloader)

In [ ]:
images, ids, labels, captions, stops = batch
captions = torch.from_numpy(captions)
stops = torch.from_numpy(stops)
images.size(), len(ids), labels.size(), captions.size(), stops.size()

In [ ]:
stops

In [ ]:
captions.long()

# Load model

In [ ]:
%run ../models/checkpoint/__init__.py
%run ../utils/files.py

In [ ]:
run_id = RunId('1215_174443', debug=False, task='cls')

In [ ]:
compiled_model = load_compiled_model(run_id)
compiled_model.metadata['model_kwargs']

# Load data

In [ ]:
%run ../datasets/__init__.py

In [ ]:
dataset_kwargs = {
    'dataset_name': 'cxr14',
    'dataset_type': 'train',
    'max_samples': None,
}
dataloader = prepare_data_classification(**dataset_kwargs)
dataset = dataloader.dataset
len(dataset)

# Do something ...